In [1]:
# import os

# # Install java
# ! apt-get update -qq
# ! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ! java -version

# # Install pyspark
# ! pip install --ignore-installed -q pyspark==2.4.4
# ! pip install --ignore-installed -q spark-nlp==2.6.3-rc1

In [2]:
import sparknlp

spark = sparknlp.start()

# params =>> gpu=False, spark23=False (start with spark 2.3)

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

Spark NLP version 2.6.4
Apache Spark version: 2.4.4


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/CS657/project/data/'

In [4]:
import datetime
import sparknlp
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import UniversalSentenceEncoder, SentimentDLModel
from sparknlp.pretrained import PretrainedPipeline 
from pyspark.sql.types import StringType, DoubleType
from pyspark.mllib.evaluation import MulticlassMetrics
import  pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
#custom pipeline with pre-trained models

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# other pre-trained sentiment classification model: sentimentdl_use_twitter
# the classes/labels/categories are in category column
sentimentdl = SentimentDLModel.pretrained(name="sentimentdl_use_imdb", lang="en")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("sentiment")
pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12.1 MB
[OK!]


In [6]:
pipeline_model = pipeline.fit([]) # already trained
pipeline_model

PipelineModel_d99ecbaac171

In [7]:
# pipeline = PretrainedPipeline('analyze_sentimentdl_use_imdb', 'en')
# pipeline = PretrainedPipeline('analyze_sentimentdl_use_twitter', 'en')

In [8]:
# yelp_review = spark.read.json(path+'/review.json')
yelp_review = spark.read.json('data/yelp_academic_dataset_review.json')
yelp_review = yelp_review.drop('date')

# reducing the dataset for now
yelp_review, _ = yelp_review.randomSplit(weights=[0.001, 0.999], seed=1) 
_.unpersist()

DataFrame[business_id: string, cool: bigint, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [9]:
# need to get sentiment for all data (train and test)
# yelp_review_annotated = pipeline.annotate(yelp_review, 'text')
yelp_review_annotated = pipeline_model.transform(yelp_review)
yelp_review.unpersist()

DataFrame[business_id: string, cool: bigint, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [10]:
yelp_review_annotated = yelp_review_annotated.withColumn('stars_thresh', yelp_review_annotated.stars - 2.5)
yelp_review_annotated = yelp_review_annotated.withColumn('stars_thresh_abs',F.abs(yelp_review_annotated.stars_thresh))
yelp_review_annotated = yelp_review_annotated.withColumn('stars_sent', yelp_review_annotated.stars_thresh / yelp_review_annotated.stars_thresh_abs)\
                                 .drop('stars_thresh_abs', 'stars_thresh')

In [11]:
# This is for discrete results
def sent(r):
    return r[0][3]

# def sent(r):
#     meta = r[0][4]
#     return float(meta['positive']) - float(meta['negative'])


def sent_to_num(r):
    if r == 'positive':
        return 1.0
    else:
        return -1.0

sent_udf = F.udf(sent)
sent_to_num_udf = F.udf(sent_to_num)

In [12]:
# This is for discrete results
yelp_review_annotated = yelp_review_annotated.withColumn('sent', sent_udf(yelp_review_annotated.sentiment))
yelp_review_annotated = yelp_review_annotated.withColumn('sent_to_num', sent_to_num_udf(yelp_review_annotated.sent))
yelp_review_annotated = yelp_review_annotated.withColumn('sent_to_num', yelp_review_annotated.sent_to_num.cast(DoubleType()))

DataFrame[business_id: string, cool: bigint, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence_embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentiment: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, stars_sent: double, sent: string, sent_to_num: double]

In [13]:
#use the metric evaluator for dataframes

predictionAndLabels = yelp_review_annotated.select('sent_to_num', 'stars_sent')
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", 
                                       predictionCol="sent_to_num",
                                       labelCol="stars_sent")
print("accuracy = ",evaluator.evaluate(predictionAndLabels))

accuracy =  0.8542141230068337


In [14]:
# IMDB
# accuracy = 0.858851069845995

# Twitter
# accuracy =  0.7484181219944318